In [11]:
from jl_song_data import SongData 
import pandas as pd 
from jl_chord_cleaning import ChordDatasetCleaner

In [12]:
data = SongData('../data/chords_clean_1_2_3.csv')

In [13]:
df = data.df

In [14]:
df

,url,name,decade,genre,chords,uuid
0,https://tabs.ultimate-guitar.com/tab/1055161,Time To Say Goodbye Con Te Partirò,1990s,Pop%%Classical%%Pop,"['G', 'D', 'Em', 'C', 'G', 'D', 'Em', 'C', 'G'...",d42333bf-e925-4ad1-a4b9-bfe2e2df209e
1,https://tabs.ultimate-guitar.com/tab/1060259,El Mañana (ver 3),2000s,Electronic%%Pop,"['Am', 'Em/G', 'F', 'Em', 'Bm', 'Dm', 'Am', 'E...",d6123fcf-d023-4425-8644-0a04afd88ebf
2,https://tabs.ultimate-guitar.com/tab/1061244,Balladen Om Herr Fredrik Åkare Och Den Söta Fr...,1960s,Jazz,"['Dm', 'E', 'Am', 'Dm', 'G7', 'C', 'E7', 'Am',...",8bc3d690-78cd-4655-add3-062e7f9df1a7
3,https://tabs.ultimate-guitar.com/tab/10cc/drea...,Dreadlock Holiday,1970s,Pop,"['Em', 'D', 'C', 'Em', 'Am', 'Em', 'Am', 'Em',...",40d74cf2-a026-4497-8b7a-08cd6527f009
4,https://tabs.ultimate-guitar.com/tab/10cc/im_n...,Im Not In Love,1970s,Pop%%Pop%%Pop,"['F#m7/B', 'B6', 'F#m7/B', 'B6', 'F#m7/B', 'B6...",277077be-828c-46ab-93c0-5ed82f514e3b
...,...,...,...,...,...,...
8939,https://tabs.ultimate-guitar.com/tab/ziggy_mar...,Lighthouse,2010s,Reggae,"['C', 'F', 'C', 'G', 'C', 'C', 'F', 'C', 'G', ...",fda4b7b8-e3bb-4c38-b5a7-73762c8c8261
8940,https://tabs.ultimate-guitar.com/tab/ziggy_mar...,Personal Revolution,2010s,Reggae,"['Bm', 'A', 'Bm', 'A', 'Bm', 'A', 'Em', 'G', '...",089f0438-4e4d-44dd-83e5-5dd929753f0f
8941,https://tabs.ultimate-guitar.com/tab/ziggy_mar...,True To Myself,2000s,Reggae%%Reggae,"['A', 'E', 'Bm', 'D', 'A', 'E', 'Bm', 'D', 'A'...",c0d75c52-ae53-4b47-bc8e-a723ba542d00
8942,https://tabs.ultimate-guitar.com/tab/ziggy_mar...,True To Myself (ver 2),2000s,Reggae,"['A', 'E', 'Bm', 'A', 'E', 'Bm', 'A', 'E', 'Bm...",29047e58-5391-4875-ab94-3a5a7fdbcd0e


In [20]:
df['chords'] = df['chords'].apply(eval)

In [21]:
def get_all_chords(dataframe):
    all_chords = []

    for chord_list in dataframe['chords']:
        all_chords.extend(chord_list)
    
    return all_chords

In [22]:
all_chords = get_all_chords(df)
len(all_chords)

779820

In [23]:
def get_unique_chords(dataframe):
    unique_chords = set([])

    for chord_list in dataframe['chords']:
        unique_chords = unique_chords.union(set(chord_list))

    unique_chords = list(unique_chords)  
    return unique_chords

In [24]:
unique_chords = get_unique_chords(df)
len(unique_chords)

1821

In [29]:
from pychord_jl import Chord
def replace_german_system(input):
    if input[0] == 'H':
        return 'Bb'+input[1:]
    
    return input

def replace_6_9(input):
    return (input
            .replace('6/9', '6_9')
            .replace('7/9', '7_9')
            .replace('7/5', '7_5')
            .replace('7/6', '7_6')
           )

def replace_enharmonic(input):
    return (input
            .replace('E#', 'F')
            .replace('B#', 'C')
            .replace('Fb', 'E')
            .replace('Cb', 'B')
           )

def replace_errors(input):
    return (input
            .replace('G(2)', 'G')
            .replace('A*', 'A')
            .replace('(III)', '')
            .replace('Gm#', 'G#m')
           )

def parse(input):
    input = replace_german_system(input)
    input = replace_6_9(input)
    input = replace_enharmonic(input)
    input = replace_errors(input)
    
    try:
        return Chord(input)
    except:
        return None

    
    

In [30]:
black_list = ['Go', 'AM']

In [31]:
parsed_chords = [ (parse(chord_str),chord_str) for chord_str in unique_chords if chord_str not in black_list]

In [32]:
def count_appearances(chord_str):
    return len([chord for chord in all_chords if chord == chord_str])

In [33]:
not_found = [ (count_appearances(x[1]),x[1]) for x in parsed_chords if x[0] is None]
len(not_found)

191

In [34]:
import functools 
total_not_found = functools.reduce(lambda x,y:x+y, [ count_appearances(x[1]) for x in parsed_chords if x[0] is None])

In [51]:
not_found

[(35, 'G6add11'),
 (21, 'Em6add9'),
 (18, 'C*'),
 (17, 'Fadd#11'),
 (16, 'CaddG'),
 (15, 'BB'),
 (14, 'G(b5)'),
 (12, 'Emmaj9'),
 (11, 'F(b5)'),
 (9, 'Cma9'),
 (9, 'Dadd4add9'),
 (8, 'C#b7b9'),
 (7, 'F+7'),
 (7, '(E)'),
 (7, 'C(2)'),
 (7, 'E7/9b'),
 (7, 'Dm(add9)/A'),
 (6, 'Am/Fnat'),
 (6, 'F#7add4'),
 (6, 'B7/13'),
 (6, 'B7/13b'),
 (6, 'D-'),
 (6, 'C(7b9)'),
 (6, 'D(2)'),
 (6, 'Bbmaj7sus4/Eb'),
 (6, 'B11b9'),
 (6, 'G(maj7)'),
 (6, 'G7+5-9'),
 (5, 'Em+'),
 (5, 'G-'),
 (5, 'C7#5#9'),
 (5, 'Bb7#5#9'),
 (5, 'C#9(#5)'),
 (5, 'E*'),
 (5, 'Ammaj9'),
 (5, 'D#(b5)'),
 (5, 'Bb-'),
 (4, 'Cm#'),
 (4, 'Bb6a'),
 (4, 'Fm7#5'),
 (4, 'Gm7b5b9'),
 (4, 'Gmaj7sus4/C'),
 (4, 'FmMaj7'),
 (4, 'F9/11'),
 (4, 'F(5)'),
 (4, 'F#dim(F)'),
 (4, 'B7#5#9'),
 (4, 'D(b5)'),
 (4, 'Em7(VII)'),
 (4, 'G7s4'),
 (4, 'Bbmaj7+5'),
 (4, 'B-'),
 (4, 'Eaddb13'),
 (3, 'E7*'),
 (3, 'Bmadd4'),
 (3, 'Cmin6'),
 (3, 'Emaj7b13/Ab'),
 (3, 'Bm*'),
 (3, 'LA'),
 (3, 'G#madd6'),
 (3, 'F#m7(b5)'),
 (3, 'Bbmaj9+11'),
 (3, 'Am*'),
 (3, 'Cm7/9

In [52]:
not_recognised_chords = [chord[1] for chord in not_found]
not_recognised_chords

['G6add11',
 'Em6add9',
 'C*',
 'Fadd#11',
 'CaddG',
 'BB',
 'G(b5)',
 'Emmaj9',
 'F(b5)',
 'Cma9',
 'Dadd4add9',
 'C#b7b9',
 'F+7',
 '(E)',
 'C(2)',
 'E7/9b',
 'Dm(add9)/A',
 'Am/Fnat',
 'F#7add4',
 'B7/13',
 'B7/13b',
 'D-',
 'C(7b9)',
 'D(2)',
 'Bbmaj7sus4/Eb',
 'B11b9',
 'G(maj7)',
 'G7+5-9',
 'Em+',
 'G-',
 'C7#5#9',
 'Bb7#5#9',
 'C#9(#5)',
 'E*',
 'Ammaj9',
 'D#(b5)',
 'Bb-',
 'Cm#',
 'Bb6a',
 'Fm7#5',
 'Gm7b5b9',
 'Gmaj7sus4/C',
 'FmMaj7',
 'F9/11',
 'F(5)',
 'F#dim(F)',
 'B7#5#9',
 'D(b5)',
 'Em7(VII)',
 'G7s4',
 'Bbmaj7+5',
 'B-',
 'Eaddb13',
 'E7*',
 'Bmadd4',
 'Cmin6',
 'Emaj7b13/Ab',
 'Bm*',
 'LA',
 'G#madd6',
 'F#m7(b5)',
 'Bbmaj9+11',
 'Am*',
 'Cm7/9',
 'F#6 (add9)',
 'F13(b9)',
 'F#7b9sus4',
 'Cm*',
 'Bbj7',
 'F#madd6',
 'C#m*',
 'A(maj7)',
 'DMin',
 'G6add11/B',
 'C3',
 'F(V)',
 'GMaj',
 'Emaj7 (add9)',
 'Fmaj7+11',
 'A(major)',
 'A11-9',
 'B7dim7/D#',
 'B7+5+9',
 'G#7(b5)',
 'B7**',
 'G#addb13',
 'Cmaj7+5/G#',
 'E7(b13)',
 'A(V)',
 'Am6add9',
 'Db6m',
 'F#Min',
 'Gm6/9

In [55]:
def contains_not_recognised_chord(chords):
    return any([not_recognised in chords for not_recognised in not_recognised_chords])

In [56]:
df[df['chords'].apply(contains_not_recognised_chord)]

,url,name,decade,genre,chords,uuid
87,https://tabs.ultimate-guitar.com/tab/2877527,Samba De Verão,2000s,World Music,"[F7M, F7M, Bdim7, E7(b13), Bb7M, Bbm6, Eb9, Am...",9402bff7-3758-45b1-9c4f-aa966b60963b
213,https://tabs.ultimate-guitar.com/tab/adele/don...,Dont You Remember (ver 5),2010s,Rhythm And Blues,"[G#madd6, G#m, G#m7, E, A, B, F#m, G#m, D4, D,...",430a11e8-a605-4764-938e-3986a6ed669f
270,https://tabs.ultimate-guitar.com/tab/aerosmith...,Dream On (ver 2),1970s,Rock,"[Em, Em7, Em6, Am6, Em, B7, Em, Em, Em7, Em6, ...",f4c040ab-915a-4e9a-b66b-58b0ec7a34fa
365,https://tabs.ultimate-guitar.com/tab/alexandra...,Hallelujah (ver 4),2000s,Pop,"[G, Em7, G, Em7, C, D, G, G, C, D, Em, C, D, B...",04de19e3-3815-4af8-a26b-671f16ea1463
409,https://tabs.ultimate-guitar.com/tab/all-4-one...,I Can Love You Like That (ver 2),1990s,Rhythm And Blues,"[Am, Bb7sus2, F/G, Cadd9/D, Bbmaj9+11, Bbmaj9a...",9f47a72e-d979-48af-8569-0588aae08a1d
...,...,...,...,...,...,...
8664,https://tabs.ultimate-guitar.com/tab/weird_al_...,Albuquerque,1990s,Comedy,"[F, B, Fm7, Bm7, F, F, F, F, F, F, F, F, F, F,...",bb328534-087c-4699-bd83-1a77bc345dee
8770,https://tabs.ultimate-guitar.com/tab/white_lio...,Wait,1980s,Metal,"[D, A/D, C/D, G/D, Bb/D, A, A, A7, D/A, Dm(add...",29f701e3-6a52-4d11-b075-885de15f15a7
8777,https://tabs.ultimate-guitar.com/tab/whitney_h...,Didnt We Almost Have It All,1980s,Rhythm And Blues,"[Bb, Bbmaj7, Eb, Dm7, Gm7, Cm7, Dm7, Eb, Bb, B...",f55409e6-af38-4530-b7cd-6b2bc3eeed80
8884,https://tabs.ultimate-guitar.com/tab/xavier_na...,Sag Es Laut,1990s,Rhythm And Blues%%Contemporary R&b,"[Ebmaj7/Em7b5, Fmaj7/F7, Cm7, Ebmaj7, Csus4, C...",65b72aa3-193c-4639-8d47-6feec4de061c


In [38]:
total_not_found / 779820

0.0008168551717062912

In [36]:
not_found.sort(key = lambda x: x[0], reverse = True) 

In [37]:
not_found

[(35, 'G6add11'),
 (21, 'Em6add9'),
 (18, 'C*'),
 (17, 'Fadd#11'),
 (16, 'CaddG'),
 (15, 'BB'),
 (14, 'G(b5)'),
 (12, 'Emmaj9'),
 (11, 'F(b5)'),
 (9, 'Cma9'),
 (9, 'Dadd4add9'),
 (8, 'C#b7b9'),
 (7, 'F+7'),
 (7, '(E)'),
 (7, 'C(2)'),
 (7, 'E7/9b'),
 (7, 'Dm(add9)/A'),
 (6, 'Am/Fnat'),
 (6, 'F#7add4'),
 (6, 'B7/13'),
 (6, 'B7/13b'),
 (6, 'D-'),
 (6, 'C(7b9)'),
 (6, 'D(2)'),
 (6, 'Bbmaj7sus4/Eb'),
 (6, 'B11b9'),
 (6, 'G(maj7)'),
 (6, 'G7+5-9'),
 (5, 'Em+'),
 (5, 'G-'),
 (5, 'C7#5#9'),
 (5, 'Bb7#5#9'),
 (5, 'C#9(#5)'),
 (5, 'E*'),
 (5, 'Ammaj9'),
 (5, 'D#(b5)'),
 (5, 'Bb-'),
 (4, 'Cm#'),
 (4, 'Bb6a'),
 (4, 'Fm7#5'),
 (4, 'Gm7b5b9'),
 (4, 'Gmaj7sus4/C'),
 (4, 'FmMaj7'),
 (4, 'F9/11'),
 (4, 'F(5)'),
 (4, 'F#dim(F)'),
 (4, 'B7#5#9'),
 (4, 'D(b5)'),
 (4, 'Em7(VII)'),
 (4, 'G7s4'),
 (4, 'Bbmaj7+5'),
 (4, 'B-'),
 (4, 'Eaddb13'),
 (3, 'E7*'),
 (3, 'Bmadd4'),
 (3, 'Cmin6'),
 (3, 'Emaj7b13/Ab'),
 (3, 'Bm*'),
 (3, 'LA'),
 (3, 'G#madd6'),
 (3, 'F#m7(b5)'),
 (3, 'Bbmaj9+11'),
 (3, 'Am*'),
 (3, 'Cm7/9

In [63]:
def any_not_found(chords):
    return len([chord for chord in not_found if chord[1] in chords]) > 0

In [64]:
any_not_found(['A'])

False

In [65]:
any_not_found(['Go'])

False

In [66]:
len(df[df['chords'].apply(lambda x: any_not_found(x))]['url'])

108

'https://tabs.ultimate-guitar.com/tab/fugees/killing-me-softly-chords-50470'

In [68]:
Chord('A7/G').components()

['G', 'A', 'C#', 'E', 'G']